# Project aim

The aim of the project is to assess the performance of an UNet or any other Deep model for cell segmentation.

In [ ]:
!conda install -n pytorchcuda ipykernel --update-deps --force-reinstall
import os
import torch

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
from torch import nn

import cv2

from torch.autograd import Variable
import torch.nn 
from torch.optim import Adam, SGD
import torch.nn.functional as F
import zipfile

import random

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

: 

In [ ]:
def apply_transform(img):
  """Applies image transformations.

  Args:
      img (PIL Image): The input image to be transformed.

  Returns:
      torch.Tensor: The transformed image as a PyTorch tensor.
  """

  # Ensure input is a PIL Image
  if not isinstance(img, transforms.Image):
    raise ValueError("Input must be a PIL Image object.")

  # Create a PyTorch transform composed of resize, normalize, and ToTensor
  transform = transforms.Compose([
      transforms.Resize(256),  # Resize to 256x256
      transforms.ToTensor(),   # Convert to PyTorch tensor
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
  ])

  # Apply the transformation to the image
  transformed_img = transform(img)

  return transformed_img

In [ ]:
class LoadDataSet(Dataset):
        def __init__(self,path, transform=None):
            self.path = path
            self.folders = os.listdir(path)
            self.transforms = apply_transform()
        
        def __len__(self):
            return len(self.folders)
              
        
        def __getitem__(self,idx):
            image_folder = os.path.join(self.path,self.folders[idx],'images/')
            mask_folder = os.path.join(self.path,self.folders[idx],'masks/')
            image_path = os.path.join(image_folder,os.listdir(image_folder)[0])
            
            img = io.imread(image_path)[:,:,:3].astype('float32')
            img = transform.resize(img,(128,128))
            
            mask = self.get_mask(mask_folder, 128, 128 ).astype('float32')

            augmented = self.transforms(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
            mask = mask[0].permute(2, 0, 1)
            return (img,mask) 


        def get_mask(self,mask_folder,IMG_HEIGHT, IMG_WIDTH):
            mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
            for mask_ in os.listdir(mask_folder):
                    mask_ = io.imread(os.path.join(mask_folder,mask_))
                    mask_ = transform.resize(mask_, (IMG_HEIGHT, IMG_WIDTH))
                    mask_ = np.expand_dims(mask_,axis=-1)
                    mask = np.maximum(mask, mask_)
              
            return mask

In [ ]:
class UNet(nn.Module):

    def __init__(self, in_channels, num_classes):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes

        # Contraction path
        self.c1 = self._contracting_block(in_channels, 16)
        self.p1 = nn.MaxPool2d(kernel_size=2)
        self.c2 = self._contracting_block(16, 32)
        self.p2 = nn.MaxPool2d(kernel_size=2)
        self.c3 = self._contracting_block(32, 64)
        self.p3 = nn.MaxPool2d(kernel_size=2)
        self.c4 = self._contracting_block(64, 128)
        self.p4 = nn.MaxPool2d(kernel_size=2)
        self.c5 = self._contracting_block(128, 256)

        # Expansive path
        self.u6 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.c6 = self._expanding_block(256, 128)
        self.u7 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.c7 = self._expanding_block(128, 64)
        self.u8 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.c8 = self._expanding_block(64, 32)
        self.u9 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.c9 = self._expanding_block(32, 16)

        # Output layer
        self.outputs = nn.Conv2d(16, num_classes, kernel_size=1)

    def _contracting_block(self, in_filters, out_filters):
        """
        Single contracting block with convolutional layers, ReLU activation,
        dropout, and max pooling.
        """
        return nn.Sequential(
            nn.Conv2d(in_filters, out_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1),
            nn.Conv2d(out_filters, out_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def _expanding_block(self, in_filters, out_filters):
        """
        Single expanding block with transposed convolution, concatenation,
        convolutional layers, ReLU activation, and dropout.
        """
        return nn.Sequential(
            nn.ConvTranspose2d(in_filters, out_filters, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Concatenate(dim=1),
            nn.Conv2d(out_filters * 2, out_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
            nn.Conv2d(out_filters, out_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        """
        Forward pass through the U-Net.
        """
        c1 = self.c1(x)
        p1 = self.p1(c1)
        c2 = self.c2(p1)
        p2 = self.p2(c2)
        c3 = self.c3(p2)
        p3 = self.p3(c3)
        c4 = self.c4(p3)
        p4 = self.p4(c4)
        c5 = self.c5(p4)

        u6 = self.u6(c5)
        cat6 = torch.cat([u6, c4], dim=1)  # Concatenate along channel dimension (dim=1)
        c6 = self.c6(cat6)
        u7 = self.u7(c6)
        cat7 = torch.cat([u7, c3], dim=1)
        c7 = self.c7(cat7)
        u8 = self.u8(c7)
        cat8 = torch.cat([u8, c2], dim=1)
        c8 = self.c8(cat8)
        u9 = self.u9(c8)
        cat9 = torch.cat([u9, c1], dim=1)
        c9 = self.c9(cat9)

        outputs = self.outputs(c9)

        return outputs


model = UNet(3, 2)  # Input channels=3 (RGB), Output classes=2
# x = torch.randn(1, 3, 32, 32)  # Example input tensor
# y = model(x)
# print(y.shape)



In [ ]:
import matplotlib.pyplot as plt

def visualize_model_layers(model):
  # Iterate through model layers
  for i, layer in enumerate(model.modules()):
    # Extract layer details
    layer_type = type(layer).__name__
    input_shape = layer.in_channels, layer.kernel_size[0], layer.kernel_size[1]  # Assuming Conv2d layer
    output_shape = layer.out_channels, layer.kernel_size[0], layer.kernel_size[1]  # Assuming Conv2d layer
    # ... (extract other relevant properties)

    # Create visual representation using Matplotlib/Seaborn
    plt.subplot(..., i+1)  # Adjust layout based on number of layers
    # ... (code to create visual elements for layer type, shapes, etc.)

  plt.show()

# ... (your model definition)

visualize_model_layers(model)
